In [6]:
import fractions
c = "3  1/2"
def mixed_to_float(x):
    return float(sum(fractions.Fraction(term) for term in x.split()))
mixed_to_float("3.5")

3.5

In [78]:
import json
path = 'recipeDB.json'
with open(path) as f:
    # load the data
    json_data = json.load(f)

indian_dict = json_data["anyToIndian"]
style_dict = json_data["styles"]

In [70]:
from bs4 import BeautifulSoup

import requests


class RecipeFetcher:

    search_base_url = 'https://www.allrecipes.com/search/results/?wt=%s&sort=re'

    def search_recipes(self, keywords): 
        search_url = self.search_base_url %(keywords.replace(' ','+'))

        page_html = requests.get(search_url)
        page_graph = BeautifulSoup(page_html.content,features = "lxml")

        return [recipe.a['href'] for recipe in\
               page_graph.find_all('div', {'class':'grid-card-image-container'})]

    def scrape_recipe(self, recipe_url):
        results = {}

        page_html = requests.get(recipe_url)
        page_graph = BeautifulSoup(page_html.content,features = "lxml")
        results['ingredients'] = [ingredient.text for ingredient in\
                                  page_graph.find_all('span', {'itemprop':'recipeIngredient'})]
        
        results['labels'] = [ingredient.text for ingredient in\
#                                   page_graph.find_all('span', {'itemprop':'recipeIngredient'})
                                  page_graph.find_all('span', {'data-id':'0'})]
        
        results['directions'] = [direction.text.strip() for direction in\
                                 page_graph.find_all('span', {'class':'recipe-directions__list--item'})
                                 if direction.text.strip()]
        for i in results['ingredients']:
            if i in results['labels']:
                results['ingredients'].remove(i)
                
        return results

    
rf = RecipeFetcher()
paneer_makhani = rf.search_recipes('mexican pizza')[0]
recipe = (rf.scrape_recipe(paneer_makhani))
recipe

{'ingredients': ['1/2 (16 ounce) can spicy fat-free refried beans*',
  '1 cup salsa, divided',
  '1 (12 inch) pre-baked Italian pizza crust',
  '2 cups shredded hearts of romaine lettuce',
  '3 medium green onions, thinly sliced',
  '1/4 cup ranch dressing',
  '1/4 cup crumbled tortilla chips',
  '1 cup shredded pepper Jack or Monterey Jack cheese'],
 'labels': [],
 'directions': ["Adjust oven rack to lowest position, and heat oven to 450 degrees. Mix beans and 1/2 cup salsa in a medium bowl. Place crust on a cookie sheet, then spread the bean mixture over crust. Bake until it's crisp and warm, about 10 minutes.",
  'Remove from oven; top with lettuce, green onions and dollop with the remaining salsa. Drizzle (or, if dressing has an easy-pour top, squirt) dressing over pizza. Top with chips and cheese, then return to oven and bake until the cheese melts, about 2 minutes longer. Cut into 6 slices and serve.']}

In [71]:
import re
dic = {}

for i in (recipe['ingredients']):
    dic[i] ={}
    x=re.findall('\w+ed ',i)
    if x!=[]:
        dic[i] = {'Preparation':x}
        
for i in (recipe['ingredients']):
    #print(i)
    if '(' in i and '%' not in i:
        x=re.findall('\(\d+ (\w+)\)',i)
        if x!=[]:
            dic[i]['Measurement'] = x
    elif '%' in i:
        x=re.findall('\d+ (\w+)',i)
        if x!=[]:
            dic[i]['Measurement'] = x
    else:
        x = re.findall('\d+ (\w+) \w+',i)
        if x!=[]:
            dic[i]['Measurement']=x
            
for i in (recipe['ingredients']):
    if '(' in i and '%' not in i:
        x=re.findall('\((\d+) \w+\)',i)
        if x!=[]:
            dic[i]['Quantity'] = mixed_to_float(x[0])
    elif '%' in i:
        x=re.findall('(\d+) \w+',i)
        if x!=[]:
            dic[i]['Quantity'] = mixed_to_float(x[0])
    else:    
        if '/' in i:
            x=re.findall('(\d*\s*\d\/\d)+',i)
            if x!=[]:
                dic[i]['Quantity'] = mixed_to_float(x[0])

        else:
            x = re.findall('(\d+) \w',i)
            if x!=[]:
                dic[i]['Quantity']=mixed_to_float(x[0])
                
for i in (recipe['ingredients']):
    x = i
    if ')' in i:
        
        x=re.sub('.*\(.*\) \w+','',x)
#         if x!=[]:
#             dic[i]['Quantity'] = x    
    if '/' in i:
            x=re.sub('\/','',x)
#             if x!=[]:
#                 dic[i]['Quantity'] = mixed_to_float(x[0])
    x =re.sub('\d+','',x)
    try:
        x = re.sub(dic[i]['Measurement'][0],'',x)
        x = re.sub(dic[i]['Preparation'][0],'',x)
    except:
        x = x
    x = re.sub('\,.*|to taste','',x).lstrip(' ')
    dic[i]['Ingredient Name'] = x


In [72]:
dic

{'1/2 (16 ounce) can spicy fat-free refried beans*': {'Preparation': ['refried '],
  'Measurement': ['ounce'],
  'Quantity': 16.0,
  'Ingredient Name': 'spicy fat-free beans*'},
 '1 cup salsa, divided': {'Measurement': ['cup'],
  'Quantity': 1.0,
  'Ingredient Name': 'salsa'},
 '1 (12 inch) pre-baked Italian pizza crust': {'Preparation': ['baked '],
  'Measurement': ['inch'],
  'Quantity': 12.0,
  'Ingredient Name': '-Italian pizza crust'},
 '2 cups shredded hearts of romaine lettuce': {'Preparation': ['shredded '],
  'Measurement': ['cups'],
  'Quantity': 2.0,
  'Ingredient Name': 'hearts of romaine lettuce'},
 '3 medium green onions, thinly sliced': {'Measurement': ['medium'],
  'Quantity': 3.0,
  'Ingredient Name': 'green onions'},
 '1/4 cup ranch dressing': {'Measurement': ['cup'],
  'Quantity': 0.25,
  'Ingredient Name': 'ranch dressing'},
 '1/4 cup crumbled tortilla chips': {'Preparation': ['crumbled '],
  'Measurement': ['cup'],
  'Quantity': 0.25,
  'Ingredient Name': 'tortilla

In [73]:
arr =[]
arr2 =[]
steps=[]

def to_Indian(recipe_dic):
    transformed_directions =[]
    for i in recipe_dic:
        curr = recipe_dic[i]['Ingredient Name'].lower()
        #print(dic[i]['Ingredient Name'])
        for a in style_dict:
            if a in curr:
                k = curr.replace(a, 'indian')
                curr = k
        for j in indian_dict:
            if (j['other']) in curr:
                curr = curr.replace(j['other'], j['indian'])
        recipe_dic[i]['Ingredient Name'] = curr
        arr.append(curr)
    while '' in arr: arr.remove('')
    print("INGREDIENTS:")
    print(arr)
    print("______________________________________________________")
    for k in recipe['directions']:
        fl = False
        j = k.lower()   
        for a in style_dict:
            if a in j:
                k = j.replace(a, 'indian')
                j = k
        for a in indian_dict:
            if a['other'] in j:
                k = j.replace(a['other'], a['indian'])
                j = k
                #print(a['other'])
                #print(k)
        transformed_directions.append(k)
        
    for k in transformed_directions:
        fl = False
        j = k.lower()   
        for a in indian_dict:   
            for word in a['other'].split():
                if word in j:
                    k = j.replace(word, a['indian'])
                    j = k
                    #print(word)
                    #print(k)
        steps.append(k)
    #print("DIRECTIONS:")
    #print(steps)
    return steps

In [74]:
to_Indian(dic)
dic

INGREDIENTS:
['spicy fat-free beans*', 'tomato puree', '-indian garlic naan', 'cabbage', 'green onions', 'pudina chutney', 'wheat roti chips', 'pepper jack or paneer']
______________________________________________________
DIRECTIONS:
["adjust oven rack to lowest position, and heat oven to 450 degrees. mix beans and 1/2 cup tomato puree in a medium bowl. place garlic naan on a cookie sheet, then spread the bean mixture over garlic naan. bake until it's crisp and warm, about 10 minutes.", 'remove from oven; top with cabbage, green onions and dollop with the remaining tomato puree. drizzle (or, if pudina chutney has an easy-pour top, squirt) pudina chutney over garlic naan. top with chips and paneer, then return to oven and bake until the paneer melts, about 2 minutes longer. cut into 6 slices and serve.']


{'1/2 (16 ounce) can spicy fat-free refried beans*': {'Preparation': ['refried '],
  'Measurement': ['ounce'],
  'Quantity': 16.0,
  'Ingredient Name': 'spicy fat-free beans*'},
 '1 cup salsa, divided': {'Measurement': ['cup'],
  'Quantity': 1.0,
  'Ingredient Name': 'tomato puree'},
 '1 (12 inch) pre-baked Italian pizza crust': {'Preparation': ['baked '],
  'Measurement': ['inch'],
  'Quantity': 12.0,
  'Ingredient Name': '-indian garlic naan'},
 '2 cups shredded hearts of romaine lettuce': {'Preparation': ['shredded '],
  'Measurement': ['cups'],
  'Quantity': 2.0,
  'Ingredient Name': 'cabbage'},
 '3 medium green onions, thinly sliced': {'Measurement': ['medium'],
  'Quantity': 3.0,
  'Ingredient Name': 'green onions'},
 '1/4 cup ranch dressing': {'Measurement': ['cup'],
  'Quantity': 0.25,
  'Ingredient Name': 'pudina chutney'},
 '1/4 cup crumbled tortilla chips': {'Preparation': ['crumbled '],
  'Measurement': ['cup'],
  'Quantity': 0.25,
  'Ingredient Name': 'wheat roti chips'},


In [75]:
import numpy as np

primary_cooking = json_data["methods"]["primary"]

primary_methods = []
for k in recipe['directions']:
    fl = False
    j = k.lower()   
    for a in primary_cooking:
        if a in j:
            primary_methods.append(a)
                
print(np.unique(primary_methods))

['bake' 'mix']


In [76]:
import numpy as np

tools_db = json_data["tools"]

tools = []
for k in recipe['directions']:
    j = k.lower()   
    for a in tools_db:
        if a in j:
            tools.append(a)
                
print(np.unique(tools))

['bowl' 'cup' 'oven']


In [77]:
dic

{'1/2 (16 ounce) can spicy fat-free refried beans*': {'Preparation': ['refried '],
  'Measurement': ['ounce'],
  'Quantity': 16.0,
  'Ingredient Name': 'spicy fat-free beans*'},
 '1 cup salsa, divided': {'Measurement': ['cup'],
  'Quantity': 1.0,
  'Ingredient Name': 'tomato puree'},
 '1 (12 inch) pre-baked Italian pizza crust': {'Preparation': ['baked '],
  'Measurement': ['inch'],
  'Quantity': 12.0,
  'Ingredient Name': '-indian garlic naan'},
 '2 cups shredded hearts of romaine lettuce': {'Preparation': ['shredded '],
  'Measurement': ['cups'],
  'Quantity': 2.0,
  'Ingredient Name': 'cabbage'},
 '3 medium green onions, thinly sliced': {'Measurement': ['medium'],
  'Quantity': 3.0,
  'Ingredient Name': 'green onions'},
 '1/4 cup ranch dressing': {'Measurement': ['cup'],
  'Quantity': 0.25,
  'Ingredient Name': 'pudina chutney'},
 '1/4 cup crumbled tortilla chips': {'Preparation': ['crumbled '],
  'Measurement': ['cup'],
  'Quantity': 0.25,
  'Ingredient Name': 'wheat roti chips'},
